# Introduction

This notebook shows how to simulate the motion of an arm with momentum and gear backlash (gear slop).  The model will require that we understand several physical processes. If you have learned the basics of Physics like $ F = ma $, you will notice that these equations are analogous. Here are some definitions: 

* $ \tau $ is torque, the angular equivalent of force ($F$). In rotational mechanics, it's defined as the force perpindicular to the radius times that radius. 
* $ \theta $ is the angle, analogous to position, ($x$).
* $ \omega $ is angular velocity (derivative of $\theta$)
* $ \alpha $ is the angular acceleration (second derivative of $\theta$)
* $ I $ is the "moment of inertia," and is analogous to mass. When multipled by angular velocity, you get angular inertia ($I \omega$), which is analogous to inertia ($m v$).



To model the movemenet of an arm with moment of inertia $ I $ and torques applied from the motor, friction, and gravity, we first need a good way to compute the torques. In the following sections, we will describe the torque equations.


## Torque From Gravity

Let's start with the torque due to gravity. Begin with the definition that the angle, $\theta$, is defined so that horizontal is zero and angle increases as the arm rises towards $ \theta = 90 ° $ when the arm is directly vertical.  We're going to have to make this a little more complicated to account for the elbow angle, but for now, let's just assume the arm has a single mass. 

$ \tau_{g} = - r\, m\, g\, \cos(\theta_s)$

Where $ \tau_g $ is the torque due to gravity. 
Arm Gravity Physics.drawio (1).svg

So let's experiment with this model alone, briefly.  Suppose the mass of the arm is $0.1 \; kg$ and its moment of inertia is $6500 \; cm^4$. 

if 

$ \Sigma \tau = I \alpha $

then 

$ \alpha = {\Sigma \tau } / I$ 

$ \frac{d^2 \theta}{d t^2} \equiv \ddot \theta \equiv α $

In [1]:
from scipy.integrate import odeint, solve_ivp
from bokeh.models import Range1d
from bokeh.models.axes import LinearAxis
from bokeh.layouts import row, column

import numpy as np
from math import pi
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()


Loading BokehJS ...

In [11]:
from basic_physics import ArmPhysics

phys = ArmPhysics()

# This function defines the derivative and acceleration given position and derivative
def armfunc(t, y):
  theta, omega = y
  kw = {"theta": theta, "omega": omega}
  # Sum all the taus we compute.  
  sigmatau = phys.torque(**kw)
  # if F = ma , then a = F / m (or \alpha = \tau / I)

  # Change in theta is just omega times dt. Change in omega is total torque / moment of inertia
  dydt = [omega, sigmatau / phys.I ] 
  return dydt 


# Initial conditions: Angle is 0 and its derivative is 0;
y0 = [30, 400]
tmax=5
t = np.linspace(0, tmax, 200)
res = solve_ivp(armfunc, [0, tmax], y0, 
                method='LSODA', 
                t_eval=t,
                first_step=0.1, min_step=1e-6, atol=0.1)
y = res["y"].T
t = res["t"]

fig = figure(height=200)
fig.line(t, y[:, 0], legend_label="Angle")
fig.line(t, y[:, 1], color='black', legend_label="Angle Rate")
fig.yaxis.axis_label = 'Arm angle (degrees)'
fig.xaxis.axis_label = 'Time (seconds)'
show(fig)

if not res['success']:
  warning(f"{res['message']}")

In [4]:
res.keys()
print(f"{res['message']}: {res['success']}")

The solver successfully reached the end of the integration interval.: True


## Motor Torque 

We use Go Builda and Rev Robotics motors.  The [Go Builda](https://www.gobilda.com/yellow-jacket-planetary-gear-motors/) and [Rev Robotics](https://docs.revrobotics.com/15mm/actuators/motors) technical docs describe torque and speed curves.  https://docs.revrobotics.com/15mm/actuators/motors/hd-hex-motor



## Gear Box Torque

Modeling the torque due to the motor is somewhat more complex.  The motor drives a step-down gearbox transmission with substantial gear backlash. Backlash is also sometimes called "slop." It is a range of angles near the target angle where the arm can move without feeling torque from the motor.

This [Desmos Graph](https://www.desmos.com/calculator/juzgzzvshu) shows the form of motor torque ($ \tau $ ) vs. $ \Delta \theta $, the angle deviation from the motor angle, $ \theta_m $.  In this model, we assume the gearbox and drive shaft have an effective torsional spring constant $ k $ in units of $ N m / degree $ and a gear backlash effect where zero force is applied until it reaches the backlash angle $ \theta_s $.  This model gives rise to a relatively simple equation: 

$ \tau = \mathrm{sign} (\Delta \theta) \; \mathrm{max} (k \left| \Delta\theta-\theta_{s} \right|, 0) $

However, abrupt changes to the value of a derivative are difficult for Ordinary Differential Equation solvers to integrate.  The solver attempts to estimate the derivative and winds up *obsessing* over the discontinuity. To avoid discontinuities, there's a clever approximation I worked up. 

$ \tau_{lin} = 0.95 \mathrm{tanh}\left( 0.6 \Delta\theta / \theta_{s} \right)^4 k \Delta\theta $

I tuned the constant parameters by trial and error.  See this [Desmos Graph](https://www.desmos.com/calculator/yjkhq4cq4x) to compare the two graphical forms.

In [5]:
delta_theta = np.linspace(-30, 30, 500)
theta_bl = 1.5
gearbox_k = 50
theta_bl=1.5

def backlash_torque(delta_theta):
  """Return the backlash force as a function of delta_theta 
  delta_theta is external angle - internal angle, and the torque tries to keep 
  them aligned.  
  """
  return -np.sign(delta_theta) * np.maximum(gearbox_k * (np.abs(delta_theta) - theta_bl), 0)

fig = figure(height=200, width=300)
fig.line(delta_theta, backlash_torque(delta_theta), color='black')
fig.line(delta_theta, -np.tanh(0.4*delta_theta/theta_bl)**2 *gearbox_k * delta_theta,
        color='green')
fig.xaxis.axis_label='Δθ (degrees)'
fig.yaxis.axis_label='Torque (N m)'
# fig.y_range = Range1d(-20, 20)

show(fig)

## Modeling the Motor

[Motor modeling](http://www.ece.northwestern.edu/local-apps/matlabhelp/toolbox/control/getstart/buildmo4.html) is tricky. The object below is an attempt at standardizing the relationship between motor and gearbox torque versus $ \Delta \theta $, the slop between the average internal motor/gearbox angle and the angle external to the gearbox. Tying this into the ODE problem isn't easy. 

In [6]:
from bokeh.transform import linear_cmap
from bokeh.palettes import viridis, Category20, Bokeh

from logging import warning
from motor import MotorAndGearbox

mot = MotorAndGearbox(gearbox_k=0.001)
fig = figure(height=300)
power = np.linspace(-1, 1, 5)

NT=7
cmap = Bokeh[NT+1]

for nt, motor_torque in enumerate(np.linspace(-4.2, 4.2, NT)):
  fig.line(power, mot.speed(power, motor_torque), line_width=2, color=cmap[nt], legend_label=f"{motor_torque:0.1f} Nm")

fig.yaxis.axis_label='Speed (deg/sec)'
fig.xaxis.axis_label='Power (%)'
print("Negative torques should reduce the motor speed. Positive torque increases it.")
show(fig)

Negative torques should reduce the motor speed. Positive torque increases it.


## Balanced at 90 degrees

This block shows what happens when the arm is balanced at 90 degrees. There is a **tiny** motor power applied (-0.0001), which determines the direction the arm falls.  I wanted to see how quickly it would fall without motor power.  This is one of the behaviors we could compare to the actual robot to tune this model.

In [7]:
from bokeh.models import Range1d

mot = MotorAndGearbox(gearbox_k=0.05)
mot.power = 0.01

# This function defines the derivative and acceleration given position and derivative
def armfunc(t, y):
  theta_ext, omega, theta_int = y
  kw = {"theta": theta_ext, "omega": omega, "delta_theta": theta_ext-theta_int}
  load_torque = mot.backlash_torque(kw['delta_theta'])
  dtheta_int = mot.speed(motor_torque=-load_torque)
  mot.theta_int = theta_int
  # Sum all the taus we compute.
  sigmatau = sum([x(**kw) for x in (tau_gravity, tau_friction)]) + mot.torque(theta_ext)
  # if F = ma , then a = F / m (or \alpha = \tau / I)
  # Omega is dtheta/dt. sum of torques / I is domega/dt, dtheta_dt is driven by the motor only.
  return [omega, sigmatau / I, dtheta_int] 


# Initial conditions: Angle is 0 and its derivative is 0;
y0 = [90, 0, 89]
t = np.linspace(0, 5, 500)
out = solve_ivp(armfunc, [np.min(t), np.max(t)], y0, t_eval=t)
y = out['y'].T
t = out['t']

fig = figure(height=200, width=500)
fig.line(t, y[:, 2] - y[:,0], line_color='red', legend_label='Backlash')
fig.line(t, y[:,0], line_color='black', legend_label='Arm Angle')
fig.line(t, y[:,2], line_color='green', legend_label='Motor Angle')
fig.yaxis.axis_label = 'Arm angle (degrees)'
fig.xaxis.axis_label = 'Time (seconds)'
ymed = np.median(y)
# fig.y_range = Range1d(-3600+ymed,3600+ymed)

delta_theta = y[:, 0] - y[:,2]
torque = mot.backlash_torque(delta_theta)
figt = figure(height=200, width=500, x_range = fig.x_range)
figt.line(t, torque, line_color='red', legend_label='Load Torque')
figt.yaxis.axis_label = 'Torque (N m)'
figt.xaxis.axis_label = 'Time (seconds)'

show(row(fig, figt))
print(list(out.keys()))
print(f"Success: {out['success']} Message: {out['message']}")

['t', 'y', 'sol', 't_events', 'y_events', 'nfev', 'njev', 'nlu', 'status', 'message', 'success']
Success: True Message: The solver successfully reached the end of the integration interval.


It takes about four seconds for the arm to swing down to -90 degrees. That seems a little faster than the robot arm, but it depends on whether the elbow is extended or retracted.  We haven't even begun to model that part of the problem yet.

## Motor Control Take 1

This is a first cut at demonstrating what happens if the motor is controlled.  I use a conditional (if t > 5) to turn off the power to the motor after 5 seconds elapses in  the IVP solver. But this doesn't work quite as expected.  The solver *does not compute values sequentially.*  It may step "backwards in time" and re-evaluate the derivatives to ensure it adequately samples the curves to get low error.  So I had to add an "else: power=1" conditional to ensure that any time we're before 5 seconds in time, the power is on. 

A better implementation would be to solve the initial value problem (IVP) in steps equal to the loop run time on the Robot Controller.  But one thing at a time.

In [8]:
def armfunc(t, y):
  # This is one way to implement the PID control.  It's probably better to run
  # solve_ivp in X millisecond steps, then update p using a PID control and 
  # simulate again.
  if t < 3: 
    mot.power=0
  else: 
    mot.power=0.2
  theta_ext, omega, theta_int = y
  kw = {"theta": theta_ext, "omega": omega, "delta_theta": theta_ext-theta_int}
  load_torque = mot.backlash_torque(kw['delta_theta'])
  dtheta_int = mot.speed(motor_torque=-load_torque)
  mot.theta_int = theta_int
  # Sum all the taus we compute.
  sigmatau = sum([x(**kw) for x in (tau_gravity, tau_friction)]) + mot.torque(theta_ext)
  # if F = ma , then a = F / m (or \alpha = \tau / I)
  # Omega is dtheta/dt. sum of torques / I is domega/dt, dtheta_dt is driven by the motor only.
  return [omega, sigmatau / I, dtheta_int] 


t = np.linspace(0, 10, 5000)
# y, other = odeint(armfunc, y0, t, tfirst=True, full_output=1,
#                   h0=1e-3, hmax=0.2, hmin=1e-4)
mot = MotorAndGearbox(free_speed=180)
mot.power = 0
mot.gearbox_k = .05
print(f"Max speed: {mot.free_speed * mot.power} deg/s")

y0 = [0, 0, 0] # External Angle, External Omega, Internal Angle.

bunch = solve_ivp(armfunc, [np.min(t), np.max(t)], y0, 
                  method='Radau', 
                  vetorized=True, 
                  t_eval=t) #, tfirst=True, full_output=1)
y = bunch['y'].T
t = bunch['t']
delta_theta = y[:, 0] - y[:,2]
theta_int = y[:, 0]
omega = y[:, 1]
kw = {"theta": theta_int, "omega": omega}

def marked_line(fig, *args, **kwargs):
  fig.circle(*args, size=2, **kwargs)
  fig.line(*args, **kwargs)

torque = mot.backlash_torque(delta_theta)

fig = figure(height=300, width=600)
fig.line(t, y[:, 2] - y[:,0], line_color='red', legend_label='Backlash')
fig.line(t, np.cos(y[:, 0]*np.pi/180), line_color='grey', legend_label='cos(theta)')
fig.yaxis.axis_label = 'Arm angle (degrees)'
fig.xaxis.axis_label = 'Time (seconds)'

fig.extra_y_ranges={
    "Angle Rate": Range1d(start=np.min(y[:,1]), end=np.max(y[:,1]))
}
fig.y_range = Range1d(start=-2, end=4)
fig.add_layout(LinearAxis(y_range_name="Angle Rate", axis_label="Angle Rate (deg/sec)"), "left")
fig.line(t, y[:, 1], line_color='green', legend_label='Arm Angle Rate', y_range_name="Angle Rate")

tfig = figure(height=300, width=600, x_range = fig.x_range)
marked_line(tfig, t, torque, color='black', line_color='black', legend_label='Motor Torque')
marked_line(tfig,t, tau_gravity(**kw), line_color='blue', legend_label='Gravity Torque')
marked_line(tfig,t, tau_friction(**kw), line_color='green', legend_label='Friction Torque')
ymed = np.median(y)
tfig.yaxis.axis_label = 'torque (N m)'
tfig.xaxis.axis_label = 'Time (seconds)'


show(column(fig ,tfig))


Max speed: 0 deg/s


c:\Users\CarlF\workspace\FTC\tis-but-a-scratch\.venv\lib\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `vetorized`.
  warn("The following arguments have no effect for a chosen solver: {}."


These graphs show the "ringing" or oscillations of the arm extremely well, but it doesn't quite reflect reality. There's a lot of parameters we could tune to try to compensate, but I'm not sure which to do just yet.  
I experimented with the cutoff time -- tryng both 4 and 5 seconds.  At 4 seconds, the backlash drops closer to 0, indicating that the arm inertia carries it across the gear backlash.  When the power to the motor is cut at 5 seconds, the backlash gets larger, indicating that the inertial of the arm is tensioning the drive shaft.  Both cases result in an under-damped oscillation, consistent with what we observe in practice.  The period of the oscillation may give insight into tuning parameters for the model too. 

In [9]:
print(t[10], y[10])
armfunc(4, y[10])

0.020004000800160033 [-4.49636288e-01 -4.48939253e+01 -3.94341813e-05]


[-44.89392531818286, -2226.3868771427774, 35.986283205987185]